From calculus we know that derivatives tell information about the local behavior of functions. That is, we can calculate the derivative of a function at a point to understand how a function changes as we move small distances away from that point.

In one dimension, the derivative tells us the slope of the tangent line. From the Taylor expansion, the derivative provides a simple linear approximation of the function: $$ f\left(x+\Delta x\right) \approx f\left(x\right) + \Delta x \, f'\left(x\right) $$

### Aside

There are three primary forms of computational derivatives.

1. Numerical differentiation. Use the limit definition of the derivative and/or Taylor series expansions to approximate the derivatives. This is particularly useful when we don't know the form of the function, and are just sampling from it. E.g., image derivatives.
2. Symbolic differentiation. Write a mathematical function out in a certain syntax, and then the program uses derivatives of simple functions and the product and chain rules to compute the derivatives of the complicated function, simplifying at the end. Done in Mathematica.
3. Automatic differentiation. Write function for a complex code in blocks, and provide the gradient for each block. This all hinges on the chain rule.

Autodiff is generalized through a computational graph. It is a graph where the nodes are operations and edges are input-output relations. Leaf nodes are input variables, and terminal nodes are outputs. Given a computational graph, two passes are made through the graph to perform autodiff.

**Forward accumulation** step makes a single forward pass through the graph. It corresponds to iteratively expanding the chain rule of the inner-most operation. Start at the source nodes: function inputs and constants. For each node, we collect the input values and the associated partial derivatives with respect to the target variable. We proceed through the tree one node at a time. At each node, we again take the inputs (as computed by the previous nodes) and compute the local partial derivatives using the previous nodes' values and partial derivatives.

To accommodate the calculation of both the node values and their derivatives, we deal not with just scalar values but also derivative values at each node. WeThese pairs are called *duals*. 

In [4]:
struct Dual
    # hold the value and the derivative
    v 
    partial 
end

# implement the base operations
# overload the operators for the Dual class
Base.:+(a::Dual, b::Dual) = Dual(a.v + b.v, a.partial + b.partial)
Base.:*(a::Dual, b::Dual) = Dual(a.v + b.v, a.v * b.partial + a.partial *b.partial)
Base.log(a::Dual) = Dual(log(a.v), a.partial/a.v) 

function Base.max(a::Dual, b::Dual)
    v = max(a.v, b.v)
    partial = a.v > b.v ? a.partial : a.v < b.v ? b.partial : NaN 
    return Dual(v, partial)
end
function Base.max(a::Dual, b::Int)
    v = max(a.v, b)
    partial = a.v > b ? a.partial : a.v < b ? 0 : NaN
    return Dual(v, partial)
end

In [5]:
using ForwardDiff


In [7]:
a = ForwardDiff.Dual(3,1)
b = ForwardDiff.Dual(2,0)
log(a*b + max(a,2))


Dual{Nothing}(2.1972245773362196,0.3333333333333333)

How does this make sense? What are the inputs, here? 

We are inputting the values $3$ and $2$ into this function and asking what the derivative of the function is at the point $\left(a,b\right)=\left(3,2\right)$. But we are also plugging in partial derivative information. What are the initial derivatives which we are plugging into this graph? From the [wikipedia article on autodiff](https://en.wikipedia.org/wiki/Automatic_differentiation#Forward_accumulation), these are called *seed values* and depend on which variable we are trying to take the derivative with respect to.

Let's work through an example.

Consider the function $$ f\left(x,y\right) = y \sin x + 2x $$ Assume we want the derivative of this with respect to $x$. Then we want to start at the input side, so we work backward in defining a computation graph: $$ \begin{align} z & = f\left(x,y\right) \\ & = y\sin x + 2x \\ & = w_2 \sin w_1 + 2w_1 \\ &= w_2 w_3 + w_4 \\ & = w5 + w4 \\ & = w6 \end{align} $$ Now we can expand the overall derivative out using the chain rule: $$ \begin{align} \frac{\partial z}{\partial x} & = \frac{\partial f}{\partial x} \\ & = \frac{\partial f}{\partial w_6}\frac{\partial w_6}{\partial x} \\ & = \frac{\partial f}{\partial w_6} \left( \frac{\partial w_5}{\partial x} + \frac{\partial w_4}{\partial x} \right) \\ & = \frac{\partial f}{\partial w_6} \left[ \left( \frac{ \partial w_5}{\partial w_2}\frac{\partial w_2}{\partial x} + \frac{\partial w_5}{\partial w_3}\frac{\partial w_3}{\partial x} \right) + \frac{\partial w_4}{\partial w_1}\frac{\partial w_1}{\partial x} \right] \\ & = \frac{\partial f}{\partial w_6} \left[ \left( \frac{ \partial w_5}{\partial w_2}\frac{\partial w_2}{\partial y}\frac{\partial y}{\partial x} + \frac{\partial w_5}{\partial w_3}\frac{\partial w_3}{\partial w_1}\frac{\partial w_1}{\partial x} \right) + \frac{\partial w_4}{\partial w_1}\frac{\partial w_1}{\partial x} \right]  \end{align} $$

Note that the derivatives which pop up in each application of the chain rule are only with respect to those variables/nodes which lead to the current node. 

Importantly, notice that at the end we have a derivative of one variable, $y$, with respect to the variable we are trying to differentiate with, $x$: $\frac{\partial y}{\partial x}$. Naturally this drops out since the variables are independent.

So the forward accumulation algorithm simply computes this expression in a particular order which allows all terms to be nicely computed. Is this dynamic programming? In the language of dual variables, the algorithm computes a dual variable at each node: these are value-derivative pairs. To kick off the algorithm, we must start at the source nodes and traverse the graph, keeping track of each node's value and the partial derivative of that node with respect to the previous nodes. Consider the case of two input nodes, $x$ and $y$, as in the current example. Since we are differentiating with respect to $x$, the initial derivatives will be $1$ and $0$, respectively. These are the seed values.

$$ \left[ s\right]$$